# Construction Loan Estimator

## Parameters

In [1]:
# All imports
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

# Global options
pd.set_option('display.max_columns', None)
output_notebook()

Loading BokehJS ...

In [12]:
# Set scenario parameters

# Initial Costs
property_cost = 85000
const_cost = 175000
down_payment = 40000
built_value = 350000
const_close_cost = 0.05
purch_cost = 400000
purch_close_cost = 0.05

# Terms
const_loan_years = 15
const_mos = 12
years_to_consider = 45
purch_loan_years = 15

# Ongoing costs
maint_per_sqft = 0.75
assess_rate = 0.07
mill_levy = 95
const_init_hoa = 150
purch_initi_hoa = 150

# Rent costs
initial_rent = 1500

# Economic factors
inflation = 0.02
rent_growth_rate = 0.035
const_apprec = 0.057
const_loan_rate = 0.04
invest_rate = 0.0992
purch_apprec = 0.057
purch_loan_rate = 0.04

# Property estimates
square_footage = 1750

# Conversions
tot_const_loan_amt = property_cost + const_cost - down_payment
const_loan_rate_mon = const_loan_rate / 12
invest_rate_mon = invest_rate / 12
rent_growth_rate_monthly = rent_growth_rate / 12
const_apprec_mon = const_apprec / 12
current_month = datetime.date.today().replace(day=1)
const_loan_mos = const_loan_years * 12
months_to_consider = years_to_consider * 12
inflation_mon = inflation / 12
purch_loan_mos = purch_loan_years * 12
purch_loan_rate_mon = purch_loan_rate / 12
purch_apprec_mon = purch_apprec / 12
tot_purch_loan_amt = purch_cost - down_payment

In [13]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, months_to_consider + 2)
monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12)
monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Overall Month'], 0)
monthly_data['Construction Loan Month'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), monthly_data['Overall Month'] - const_mos, 0)
monthly_data['Purchase Loan Month'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, monthly_data['Overall Month'], 0)

# Calculate home value
monthly_data['Build Home Value'] = np.where(monthly_data['Overall Month'] <= const_mos, 0, built_value * (e**(const_apprec_mon * (monthly_data['Overall Month'] - const_mos))))

monthly_data['Purchase Home Value'] = purch_cost * (e**(purch_apprec_mon * (monthly_data['Overall Month'])))

# Calculate loan value
monthly_data['Construction Loan Value'] = np.where(monthly_data['Construction Loan Month'] != 0, tot_const_loan_amt * (((1 + const_loan_rate_mon) ** (const_loan_mos)) - ((1 + const_loan_rate_mon)**(monthly_data['Construction Loan Month'] - 1))) / ((1 + const_loan_rate_mon)**(const_loan_mos) - 1), tot_const_loan_amt)
monthly_data['Construction Loan Value'] = np.where(monthly_data['Overall Month'] > (const_mos + const_loan_mos), 0, monthly_data['Construction Loan Value'])

monthly_data['Purchase Loan Value'] = np.where(monthly_data['Purchase Loan Month'] != 0, tot_purch_loan_amt * (((1 + purch_loan_rate_mon) ** (purch_loan_mos)) - ((1 + purch_loan_rate_mon)**(monthly_data['Purchase Loan Month'] - 1))) / ((1 + purch_loan_rate_mon)**(purch_loan_mos) - 1), tot_purch_loan_amt)
monthly_data['Purchase Loan Value'] = np.where(monthly_data['Overall Month'] > purch_loan_mos, 0, monthly_data['Purchase Loan Value'])

# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= const_mos, tot_const_loan_amt / const_mos, 0)

# Calculate monthly payments
monthly_data['Construction Mortgage Payment'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), tot_const_loan_amt * (const_loan_rate_mon * ((1 + const_loan_rate_mon)**const_loan_mos)) / ((1 + const_loan_rate_mon)**const_loan_mos - 1), 0)

monthly_data['Purchase Mortgage Payment'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, tot_purch_loan_amt * (purch_loan_rate_mon * ((1 + purch_loan_rate_mon)**purch_loan_mos)) / ((1 + purch_loan_rate_mon)**purch_loan_mos - 1), 0)

# Calculate interest payment
monthly_data['Construction Interest Payment'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Construction Disbursements'].cumsum() * const_loan_rate_mon, monthly_data['Construction Loan Value'] * const_loan_rate_mon)
monthly_data['Construction Cumulative Interest Payments'] = monthly_data['Construction Interest Payment'].cumsum()

monthly_data['Purchase Interest Payment'] = monthly_data['Purchase Loan Value'] * purch_loan_rate_mon
monthly_data['Purchase Cumulative Interest Payments'] = monthly_data['Purchase Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Construction Principal Payment'] = np.where(monthly_data['Overall Month'] > const_mos, monthly_data['Construction Mortgage Payment'] - monthly_data['Construction Interest Payment'], 0)
monthly_data['Construction Cumulative Principal Payments'] = monthly_data['Construction Principal Payment'].cumsum()

monthly_data['Purchase Principal Payment'] = monthly_data['Purchase Mortgage Payment'] - monthly_data['Purchase Interest Payment']
monthly_data['Purchase Cumulative Principal Payments'] = monthly_data['Purchase Principal Payment'].cumsum()

# Calculate property taxes
monthly_data['Construction Property Tax Payment'] = np.where(monthly_data['Overall Month'] > const_mos, (((monthly_data['Build Home Value'] * assess_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Construction Cumulative Property Taxes'] = monthly_data['Construction Property Tax Payment'].cumsum()

monthly_data['Purchase Property Tax Payment'] = (((monthly_data['Purchase Home Value'] * assess_rate) * (mill_levy / 1000)) / 12)
monthly_data['Purchase Cumulative Property Taxes'] = monthly_data['Purchase Property Tax Payment'].cumsum()

# Calculate maintenance costs
monthly_data['Construction Maintenance Costs'] = np.where(monthly_data['Overall Month'] > const_mos, ((square_footage * maint_per_sqft) / 12), 0)
monthly_data['Construction Cumulative Maintenance Costs'] = monthly_data['Construction Maintenance Costs'].cumsum()

monthly_data['Purchase Maintenance Costs'] = ((square_footage * maint_per_sqft) / 12)
monthly_data['Purchase Cumulative Maintenance Costs'] = monthly_data['Purchase Maintenance Costs'].cumsum()

# Calculate HOA costs
monthly_data['Construction HOA Costs'] = const_init_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Construction Cumulative HOA Costs'] = monthly_data['Construction HOA Costs'].cumsum()

monthly_data['Purchase HOA Costs'] = purch_initi_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Purchase Cumulative HOA Costs'] = monthly_data['Purchase HOA Costs'].cumsum()

# Calculate total home payments
monthly_data['Construction Total Home Payments'] = monthly_data['Construction Interest Payment'] + monthly_data['Construction Principal Payment'] + monthly_data['Construction Property Tax Payment'] + monthly_data['Construction Maintenance Costs'] + monthly_data['Construction HOA Costs']
monthly_data['Construction Cumulative Home Payments'] = monthly_data['Construction Total Home Payments'].cumsum() + down_payment + (const_close_cost * tot_const_loan_amt)

monthly_data['Purchase Total Home Payments'] = monthly_data['Purchase Interest Payment'] + monthly_data['Purchase Principal Payment'] + monthly_data['Purchase Property Tax Payment'] + monthly_data['Purchase Maintenance Costs'] + monthly_data['Purchase HOA Costs']
monthly_data['Purchase Cumulative Home Payments'] = monthly_data['Purchase Total Home Payments'].cumsum() + down_payment + (purch_close_cost * tot_purch_loan_amt)

# Calculate equity metrics
monthly_data['Construction Equity Ownership Proportion'] = 1- (monthly_data['Construction Loan Value'] / tot_const_loan_amt)
monthly_data['Construction Equity Value'] = monthly_data['Construction Equity Ownership Proportion'] * monthly_data['Build Home Value']
monthly_data['Construction Equity Gain'] = monthly_data['Construction Equity Value'] - monthly_data['Construction Cumulative Home Payments']
monthly_data['Construction Gains upon Sale'] = monthly_data['Build Home Value'] - monthly_data['Construction Cumulative Home Payments'] - monthly_data['Construction Loan Value']

monthly_data['Purchase Equity Ownership Proportion'] = 1- (monthly_data['Purchase Loan Value'] / tot_purch_loan_amt)
monthly_data['Purchase Equity Value'] = monthly_data['Purchase Equity Ownership Proportion'] * monthly_data['Purchase Home Value']
monthly_data['Purchase Equity Gain'] = monthly_data['Purchase Equity Value'] - monthly_data['Purchase Cumulative Home Payments']
monthly_data['Purchase Gains upon Sale'] = monthly_data['Purchase Home Value'] - monthly_data['Purchase Cumulative Home Payments'] - monthly_data['Purchase Loan Value']

# Calculate rent payments
monthly_data['Rent Payments'] = np.where(monthly_data['Overall Month'] == 1, (initial_rent + down_payment), initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1))))
monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

# Calculate rent investment value
monthly_data['Rent Investment'] = down_payment * (e**(invest_rate_mon * (monthly_data['Overall Month'] - 1)))

# Calculate when intersection points
# monthly_data['Decision'] = np.where(monthly_data['Cumulative Rent Payments'] - monthly_data['Cumulative Home Payments'] > 0, 'Buy', 'Rent')

monthly_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,4.019045e+05,220000.0,360000.000000,18333.333333,0.0,2662.876532,61.111111,61.111111,1200.000000,1200.000000,0.0,0.000000,1462.876532,1462.876532,0.000000,0.000000,222.722088,222.722088,0.000,0.000,109.375,109.375,150.000000,150.000000,150.000000,150.000000,211.111111,5.121111e+04,3144.973620,6.114497e+04,0.0,0.000000e+00,-5.121111e+04,-2.712111e+05,0.000000,0.000000e+00,-6.114497e+04,-1.924045e+04,41500.000000,4.150000e+04,4.000000e+04
1,2,0,0,2,0,2,0.000000e+00,4.038181e+05,220000.0,358537.123468,18333.333333,0.0,2662.876532,122.222222,183.333333,1195.123745,2395.123745,0.0,0.000000,1467.752787,2930.629319,0.000000,0.000000,223.782534,446.504622,0.000,0.000,109.375,218.750,150.250208,300.250208,150.250208,300.250208,272.472431,5.148358e+04,3146.284275,6.429126e+04,0.0,0.000000e+00,-5.148358e+04,-2.714836e+05,0.004064,1.640933e+03,-6.265032e+04,-1.901027e+04,1504.381386,4.300438e+04,4.033204e+04
2,3,0,0,3,0,3,0.000000e+00,4.057408e+05,220000.0,357069.370681,18333.333333,0.0,2662.876532,183.333333,366.666667,1190.231236,3585.354980,0.0,0.000000,1472.645297,4403.274616,0.000000,0.000000,224.848030,671.352652,0.000,0.000,109.375,328.125,150.500834,450.751043,150.500834,450.751043,333.834168,5.181742e+04,3147.600396,6.743886e+04,0.0,0.000000e+00,-5.181742e+04,-2.718174e+05,0.008141,3.302989e+03,-6.413587e+04,-1.876742e+04,1508.775571,4.451316e+04,4.066683e+04
3,4,0,0,4,0,4,0.000000e+00,4.076727e+05,220000.0,355596.725384,18333.333333,0.0,2662.876532,244.444444,611.111111,1185.322418,4770.677398,0.0,0.000000,1477.554114,5880.828730,0.000000,0.000000,225.918599,897.271251,0.000,0.000,109.375,437.500,150.751878,601.502921,150.751878,601.502921,395.196323,5.221261e+04,3148.922009,7.058778e+04,0.0,0.000000e+00,-5.221261e+04,-2.722126e+05,0.012231,4.986374e+03,-6.560141e+04,-1.851185e+04,1513.182590,4.602634e+04,4.100440e+04
4,5,0,0,5,0,5,0.000000e+00,4.096137e+05,220000.0,354119.171270,18333.333333,0.0,2662.876532,305.555556,916.666667,1180.397238,5951.074636,0.0,0.000000,1482.479295,7363.308025,0.000000,0.000000,226.994265,1124.265516,0.000,0.000,109.375,546.875,151.003341,752.506262,151.003341,752.506262,456.558896,5.266917e+04,3150.249138,7.373803e+04,0.0,0.000000e+00,-5.266917e+04,-2.726692e+05,0.016336,6.691300e+03,-6.704673e+04,-1.824349e+04,1517.602481,4.754394e+04,4.134478e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,537,44,0,0,0,0,4.237307e+06,5.126685e+06,0.0,0.000000,0.000000,0.0,0.000000,0.000000,78411.012184,0.000000,120508.92

In [14]:
# Shrink set to year starts for simpler graphs
year_data = monthly_data.loc[monthly_data['Year Start'] == 1]

year_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,4.019045e+05,220000.000000,360000.000000,18333.333333,0.000000,2662.876532,61.111111,61.111111,1200.000000,1200.000000,0.000000,0.000000,1462.876532,1462.876532,0.000000,0.000000,222.722088,222.722088,0.000,0.000,109.375,109.375,150.000000,150.000000,150.000000,150.000000,211.111111,5.121111e+04,3144.973620,6.114497e+04,0.000000,0.000000e+00,-5.121111e+04,-2.712111e+05,0.000000,0.000000e+00,-6.114497e+04,-1.924045e+04,41500.000000,4.150000e+04,4.000000e+04
12,13,1,1,0,1,13,3.516665e+05,4.254786e+05,220000.000000,342120.045893,0.000000,1627.313436,2662.876532,733.333333,5500.000000,1140.400153,15214.964432,893.980103,893.980103,1522.476379,19402.430486,194.881827,194.881827,235.786033,2979.563350,109.375,109.375,109.375,1421.875,153.030201,1969.636124,153.030201,1969.636124,2084.600464,5.966787e+04,3161.067766,9.898847e+04,0.000000,0.000000e+00,-5.966787e+04,7.199858e+04,0.049667,2.113205e+04,-7.785642e+04,-1.562996e+04,1553.429563,5.984544e+04,4.417149e+04
24,25,2,1,0,13,25,3.722937e+05,4.504353e+05,209073.361379,323511.634869,0.000000,1627.313436,2662.876532,696.911205,14064.700487,1078.372116,28499.043758,930.402232,11857.040853,1584.504416,38072.869547,206.312778,2607.117931,249.616253,5898.109370,109.375,1421.875,109.375,2734.375,156.121616,3826.031336,156.121616,3826.031336,2099.122831,8.477677e+04,3177.989402,1.370304e+05,0.049667,1.849054e+04,-6.628622e+04,7.844361e+04,0.101357,4.565458e+04,-9.137585e+04,-1.010672e+04,1608.762272,7.884433e+04,4.877800e+04
36,37,3,1,0,25,37,3.941309e+05,4.768560e+05,197701.554642,304145.088468,0.000000,1627.313436,2662.876532,659.005182,22182.748963,1013.816962,41022.460588,968.308254,23266.753612,1649.059571,57503.971103,218.414222,5160.845699,264.257697,8987.845073,109.375,2734.375,109.375,4046.875,159.275482,5719.928218,159.275482,5719.928218,2114.378140,1.100647e+05,3195.784711,1.752811e+05,0.101357,3.994776e+04,-7.011689e+04,8.636472e+04,0.155153,7.398541e+04,-1.012957e+05,-2.570174e+03,1666.065916,9.851996e+04,5.386492e+04
48,49,4,1,0,37,49,4.172490e+05,5.048264e+05,185866.442953,283989.519086,0.000000,1627.313436,2662.876532,619.554810,29835.948137,946.631730,52754.224361,1007.758626,35141.315674,1716.244802,77726.725716,231.225485,7864.364439,279.757946,12258.811726,109.375,4046.875,109.375,5359.375,162.493060,7652.084356,162.493060,7652.084356,2130.406981,1.355406e+05,3214.502539,2.137512e+05,0.155153,6.473724e+04,-7.080335e+04,9.584196e+04,0.211140,1.065892e+05,-1.071621e+05,7.085629e+03,1725.410698,1.188964e+05,5.948234e+04
60,61,5,1,0,49,61,4.417231e+05,5.344374e+05,173549.150552,263012.780708,0.000000,1627.313436,2662.876532,578.497169

In [15]:
# Plot payment comparison
payments = figure(title='Rent vs. Buy Cumulative Cost')
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Cost"
payments.line(year_data['Overall Year'], year_data['Cumulative Rent Payments'], line_color='red', legend_label='Renting')
payments.line(year_data['Overall Year'], year_data['Construction Cumulative Home Payments'], line_color='blue', legend_label='Building')
payments.line(year_data['Overall Year'], year_data['Purchase Cumulative Home Payments'], line_color='green', legend_label='Purchase')

show(payments)

In [16]:
# Plot returns comparison
returns = figure(title='Rent vs. Buy Cumulative Returns')
returns.xaxis.axis_label = "Year"
returns.yaxis.formatter.use_scientific = False
returns.yaxis.axis_label = "Returns"
returns.line(year_data['Overall Year'], year_data['Rent Investment'], line_color='red', legend_label='Renting')
returns.line(year_data['Overall Year'], year_data['Construction Equity Gain'], line_color='blue', legend_label='Construction Equity Gain')
returns.line(year_data['Overall Year'], year_data['Construction Gains upon Sale'], line_color='purple', legend_label='Construction Gains upon Sale')
returns.line(year_data['Overall Year'], year_data['Purchase Equity Gain'], line_color='green', legend_label='Purchase Equity Gain')
returns.line(year_data['Overall Year'], year_data['Purchase Gains upon Sale'], line_color='grey', legend_label='Purchase Gains upon Sale')
# Include line below if you wish to see the value of the home in relation to the equity value
# payments.line(year_data['Overall Year'], year_data['Home Value'], line_color='yellow', legend_label='Home Value')

show(returns)